# **Tarea 5**

# 1. Importar Bibliotecas para Ciencia de Datos <a id="2"></a>

Instalar bibliotecas.

Importar los paquetes de procesamiento y visualización de datos.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import preprocessing
from sklearn import metrics
from sklearn.model_selection import train_test_split
%matplotlib inline

In [2]:
anime_df = pd.read_csv('anime.csv')
rating_df = pd.read_csv('rating.csv')



## Pre-procesamiento de los Datos

Visualizacion rapida de los datos, para observar como se componen y que variables son las que tenemos disponibles

In [3]:
anime_df.head()


,anime_id,name,genre,type,episodes,rating,members
0,32281,Kimi no Na wa.,"Drama, Romance, School, Supernatural",Movie,1,9.37,200630
1,5114,Fullmetal Alchemist: Brotherhood,"Action, Adventure, Drama, Fantasy, Magic, Mili...",TV,64,9.26,793665
2,28977,Gintama°,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.25,114262
3,9253,Steins;Gate,"Sci-Fi, Thriller",TV,24,9.17,673572
4,9969,Gintama&#039;,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.16,151266


In [4]:
rating_df.head()

,user_id,anime_id,rating
0,1,20,-1
1,1,24,-1
2,1,79,-1
3,1,226,-1
4,1,241,-1


In [5]:
anime_df.info()
anime_df.isnull().sum()
total_nulos = anime_df.isnull().sum().sum()
print(f"Tenemos un total de {total_nulos} nulos")



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12294 entries, 0 to 12293
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   anime_id  12294 non-null  int64  
 1   name      12294 non-null  object 
 2   genre     12232 non-null  object 
 3   type      12269 non-null  object 
 4   episodes  12294 non-null  object 
 5   rating    12064 non-null  float64
 6   members   12294 non-null  int64  
dtypes: float64(1), int64(2), object(4)
memory usage: 672.5+ KB
Tenemos un total de 317 nulos


In [6]:

##Como la columna genre posee 62 nulos, debemos especificar de mejor forma que estos no poseen gener, por ende le asignaremos
## "Not genre" a estos 62 nulos.
anime_df['genre'] = anime_df['genre'].replace(np.nan, "Unknown")
anime_df['rating'] = anime_df['rating'].replace(np.nan, "Unknown")
anime_df['type'] = anime_df['type'].replace(np.nan, "Unknown")
anime_df.isnull().sum()

anime_id    0
name        0
genre       0
type        0
episodes    0
rating      0
members     0
dtype: int64

In [7]:
rating_df.info()
rating_df.isnull().sum()
total_nulos_rating = rating_df.isnull().sum().sum()
print(f"Tenemos un total de {total_nulos_rating} nulos")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7813737 entries, 0 to 7813736
Data columns (total 3 columns):
 #   Column    Dtype
---  ------    -----
 0   user_id   int64
 1   anime_id  int64
 2   rating    int64
dtypes: int64(3)
memory usage: 178.8 MB
Tenemos un total de 0 nulos


In [8]:
anime_df.duplicated().sum()
total_duplicados_anime=anime_df.duplicated().sum()
total_duplicados_rtg=rating_df.duplicated().sum().sum()

print(f"DF de anime tenemos un total de duplicados de {total_duplicados_anime}")
print(f"DF de rating tenemos un total de duplicados de {total_duplicados_rtg}")

DF de anime tenemos un total de duplicados de 0
DF de rating tenemos un total de duplicados de 1


In [9]:
###Eliminamos los duplicados del DF rating
df2 = rating_df.drop_duplicates(keep=False)
total_dup = df2.duplicated().sum()
print(f"DF de rating tenemos un total de duplicados de {total_dup} nulos")

DF de rating tenemos un total de duplicados de 0 nulos


In [10]:
##guardamos en un nuevo data frame con los generos como columnas y le asignamos 0 o 1 en la columna correspondiente
## 0  si es que no pertence 
## 1  si es que pertenece
df5 = anime_df.genre.astype(str).str.strip('[]').str.get_dummies(', ')
df5.columns = df5.columns.str.strip("'")
#Luego concatenamos los dataframe y eliminamos la columna genre 
anime_df_gen=pd.concat([anime_df, df5], axis=1)
anime_df_gen.drop('genre', axis=1, inplace=True)
##Rellenar los valores de NaN con 0
anime_df_gen = anime_df_gen.fillna(0)

anime_df_gen.head()


,anime_id,name,type,episodes,rating,members,Action,Adventure,Cars,Comedy,...,Slice of Life,Space,Sports,Super Power,Supernatural,Thriller,Unknown,Vampire,Yaoi,Yuri
0,32281,Kimi no Na wa.,Movie,1,9.37,200630,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
1,5114,Fullmetal Alchemist: Brotherhood,TV,64,9.26,793665,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2,28977,Gintama°,TV,51,9.25,114262,1,0,0,1,...,0,0,0,0,0,0,0,0,0,0
3,9253,Steins;Gate,TV,24,9.17,673572,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
4,9969,Gintama&#039;,TV,51,9.16,151266,1,0,0,1,...,0,0,0,0,0,0,0,0,0,0


# Sistema de Recomendaciones con Filtrado Basado en Contenidos <a id="6"></a>

Crearemos un usuario, el ucal tiene como registor las peliculas vistas y el rating asignado por este.

In [11]:
usuario = [
            {'title':'Under World', 'rating_user':8},
            {'title':'Fullmetal Alchemist: Brotherhood', 'rating_user':7.5},
            {'title':'Gintama', 'rating_user':4},
            {'title':"Sakura no Mori", 'rating_user':3},
            {'title':'Natsume Yuujinchou Go', 'rating_user':6.5},
            {'title':'Death Note', 'rating_user':7.5},
            {'title':'Pink no Curtain', 'rating_user':2.5}
         ] 
usuario = pd.DataFrame(usuario)
usuario

,title,rating_user
0,Under World,8.0
1,Fullmetal Alchemist: Brotherhood,7.5
2,Gintama,4.0
3,Sakura no Mori,3.0
4,Natsume Yuujinchou Go,6.5
5,Death Note,7.5
6,Pink no Curtain,2.5


In [12]:
df_usuario_anime = anime_df_gen[anime_df_gen['name'].isin(usuario['title'].tolist())]
df_usuario_anime

,anime_id,name,type,episodes,rating,members,Action,Adventure,Cars,Comedy,...,Slice of Life,Space,Sports,Super Power,Supernatural,Thriller,Unknown,Vampire,Yaoi,Yuri
1,5114,Fullmetal Alchemist: Brotherhood,TV,64,9.26,793665,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
12,918,Gintama,TV,201,9.04,336376,1,0,0,1,...,0,0,0,0,0,0,0,0,0,0
31,32983,Natsume Yuujinchou Go,TV,13,8.76,38865,0,0,0,0,...,1,0,0,0,1,0,0,0,0,0
40,1535,Death Note,TV,37,8.71,1013917,0,0,0,0,...,0,0,0,0,1,1,0,0,0,0
12278,17833,Pink no Curtain,OVA,1,3.61,138,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
12281,9504,Sakura no Mori,OVA,2,4.53,221,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
12290,5543,Under World,OVA,1,4.28,183,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [13]:
##Cambiamos el nombre de la columna name por tittle, ya que apra poder realziar un merge debemos tener almenos una columna igual 
#en los dataframe
df_usuario_anime.columns = df_usuario_anime.columns.str.replace('name', 'title')
##Eliminamos las columnas que no utilizaremos, se elimino el rating de las peliculas, se alude a esto para no confundir con el rating
##de los usuarios (rating_user), que es la ultima columna del df.
df_usuario_anime=df_usuario_anime.drop(['type', 'episodes', 'members', 'rating'], axis=1)
df_usuario_anime = pd.merge(df_usuario_anime, usuario)
df_usuario_anime

,anime_id,title,Action,Adventure,Cars,Comedy,Dementia,Demons,Drama,Ecchi,...,Space,Sports,Super Power,Supernatural,Thriller,Unknown,Vampire,Yaoi,Yuri,rating_user
0,5114,Fullmetal Alchemist: Brotherhood,1,1,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,7.5
1,918,Gintama,1,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,4.0
2,32983,Natsume Yuujinchou Go,0,0,0,0,0,0,1,0,...,0,0,0,1,0,0,0,0,0,6.5
3,1535,Death Note,0,0,0,0,0,0,0,0,...,0,0,0,1,1,0,0,0,0,7.5
4,17833,Pink no Curtain,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2.5
5,9504,Sakura no Mori,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,3.0
6,5543,Under World,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,8.0


Construiremos una tabla por genero, de las peliculas vista por nuestro usuario

In [14]:

df_usuario_anime_genero=df_usuario_anime
df_usuario_anime_genero=df_usuario_anime_genero.drop(['anime_id', 'title', 'rating_user'], axis=1)
df_usuario_anime_genero

,Action,Adventure,Cars,Comedy,Dementia,Demons,Drama,Ecchi,Fantasy,Game,...,Slice of Life,Space,Sports,Super Power,Supernatural,Thriller,Unknown,Vampire,Yaoi,Yuri
0,1,1,0,0,0,0,1,0,1,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,1,0,1,0,...,1,0,0,0,1,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,1,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Luego de tener los datos procesados, estos estan listo para darle un peso a los generos segun la clasficiacion de neustro usuario.
Para esto realizaremos una operacion matematica ya conocida, como lo es el producto punto entre una matriz y un vector, para realziar esta operacion, utilizaremos el operador `dot` que nos entrega la libreria de pandas

In [15]:
# Producto punto para obtener los pesos
perfil_usuario = df_usuario_anime_genero.transpose().dot(df_usuario_anime['rating_user'])
#
# El perfil del usuario
perfil_usuario

Action           11.5
Adventure         7.5
Cars              0.0
Comedy            4.0
Dementia          0.0
Demons            0.0
Drama            14.0
Ecchi             0.0
Fantasy          14.0
Game              0.0
Harem             0.0
Hentai           13.5
Historical        4.0
Horror            0.0
Josei             0.0
Kids              0.0
Magic             7.5
Martial Arts      0.0
Mecha             0.0
Military          7.5
Music             0.0
Mystery           7.5
Parody            4.0
Police            7.5
Psychological     7.5
Romance           0.0
Samurai           4.0
School            0.0
Sci-Fi            4.0
Seinen            0.0
Shoujo            6.5
Shoujo Ai         0.0
Shounen          11.5
Shounen Ai        0.0
Slice of Life     9.0
Space             0.0
Sports            0.0
Super Power       0.0
Supernatural     14.0
Thriller          7.5
Unknown           0.0
Vampire           0.0
Yaoi              0.0
Yuri              0.0
dtype: float64

Esta operacion de botener los pesos de cada genero en base al rating asignado por el usario se lo conoce como **perfil de usuario**.
En base a este perfil podemos recomendar peliculas que cumplan los criterios de eleccion del usario.


In [16]:
df_anime_con_genero_tabla = anime_df_gen.set_index(anime_df_gen['anime_id'])
df_anime_con_genero_tabla =df_anime_con_genero_tabla.drop(['anime_id', 'name', 'type','episodes','rating','members'], axis=1)
df_anime_con_genero_tabla

,Action,Adventure,Cars,Comedy,Dementia,Demons,Drama,Ecchi,Fantasy,Game,...,Slice of Life,Space,Sports,Super Power,Supernatural,Thriller,Unknown,Vampire,Yaoi,Yuri
anime_id,,,,,,,,,,,,,,,,,,,,,
32281,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,1,0,0,0,0,0
5114,1,1,0,0,0,0,1,0,1,0,...,0,0,0,0,0,0,0,0,0,0
28977,1,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9253,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
9969,1,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9316,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5543,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5621,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [17]:
recomendacion_user = ((df_anime_con_genero_tabla*perfil_usuario).sum(axis=1))/(perfil_usuario.sum())

# Ordenar las recomendaciones en orden descendente
recomendacion_user = recomendacion_user.sort_values(ascending=False)

recomendacion_user

anime_id
231      0.537538
4938     0.525526
969      0.480480
9135     0.465465
121      0.465465
           ...   
31521    0.000000
28487    0.000000
29814    0.000000
32660    0.000000
18729    0.000000
Length: 12294, dtype: float64

In [18]:
recomedacion_top30 = anime_df.loc[anime_df['anime_id'].isin(recomendacion_user.head(15).keys())]
recomedacion_top30

,anime_id,name,genre,type,episodes,rating,members
1,5114,Fullmetal Alchemist: Brotherhood,"Action, Adventure, Drama, Fantasy, Magic, Mili...",TV,64,9.26,793665
173,6811,InuYasha: Kanketsu-hen,"Action, Adventure, Comedy, Demons, Fantasy, Ma...",TV,26,8.37,99128
200,121,Fullmetal Alchemist,"Action, Adventure, Comedy, Drama, Fantasy, Mag...",TV,51,8.33,600384
286,4938,Tsubasa: Shunraiki,"Action, Adventure, Drama, Fantasy, Magic, Myst...",OVA,2,8.23,40420
342,14075,Zetsuen no Tempest,"Action, Drama, Fantasy, Magic, Mystery, Psycho...",TV,24,8.17,259900
1051,31821,Arslan Senki (TV): Fuujin Ranbu,"Action, Adventure, Drama, Fantasy, Historical,...",TV,8,7.7,53577
1071,969,Tsubasa Chronicle 2nd Season,"Action, Adventure, Drama, Fantasy, Mystery, Ro...",TV,26,7.7,79166
1558,9135,Fullmetal Alchemist: The Sacred Star of Milos,"Action, Adventure, Comedy, Drama, Fantasy, Mag...",Movie,1,7.5,87944
3316,2832,Ani*Kuri15,"Adventure, Comedy, Drama, Fantasy, Game, Magic...",Special,15,7.02,12926
5917,231,Asagiri no Miko,"Action, Comedy, Drama, Fantasy, Magic, School,...",TV,26,6.31,4721
